# Step 1 — Threshold Rule + Anchor Quality (Behavioral Extension)

**Goal:**  
Introduce behavioral realism by adding two movement rules — the **threshold rule** and **anchor quality**.  
Customers no longer move purely at random: their decisions depend on the similarity of nearby stores and the attractiveness of the anchor.

---

**Why Step 1 matters**  
- Adds **social clustering** behavior (agents prefer similar stores).  
- Captures **anchor influence** on staying and spillover effects.  
- Moves the model beyond random movement toward *realistic retail dynamics*.

---

**Key assumptions**  
- Each agent checks the proportion of **same-category neighbors** before moving.  
- If the ratio ≥ `theta_same`, the agent moves toward a similar store.  
- Otherwise, it moves randomly or stays put.  
- Anchors have a probability `p_stay_anchor` of keeping agents longer (anchor quality).  
- Higher-quality anchors concentrate traffic, but too high a value reduces diffusion.

---

**Parameters introduced**  
| Parameter | Meaning | Example values |
|------------|----------|----------------|
| `theta_same` | Minimum ratio of same-category neighbors required to move | 0.4 = diffusion ↑ 0.8 = clustering ↑ |
| `p_stay_anchor` | Probability of staying longer at anchor (anchor quality) | 0.3 = medium 0.5 = high |

---

**Expected patterns**  
- **Higher `theta_same` →** stronger clustering, reduced exploration.  
- **Higher `p_stay_anchor` →** more footfall at the anchor and 1-hop neighbors.  
- Moderate values (e.g., `theta_same = 0.6`, `p_stay_anchor = 0.3`) give balanced diffusion and clustering.

In [ ]:
import sys, os

# Add project paths (adjust if your notebook lives elsewhere)
PROJECT_ROOT = os.path.abspath("..")
sys.path.append(PROJECT_ROOT)
sys.path.append(os.path.join(PROJECT_ROOT, "asm"))
sys.path.append(os.path.join(PROJECT_ROOT, "utils"))

print("Path added:", PROJECT_ROOT)

In [ ]:
from utils import ensure_dir            # if your ensure_dir sits in utils/__init__.py
# OR: from utils.io import ensure_dir   # if ensure_dir lives in utils/io.py
try:
    from utils.random_tools import set_seed
    from utils.plotting import set_palette
    set_seed(42)
    set_palette(style='whitegrid', context='talk', palette='Set2')
except Exception as e:
    print("Optional styling utils not found or failed:", e)

In [ ]:
from asm.simulate_step1 import simulate, Params

# Quick sanity check: show default params
Params()


## **Single run (preview)**

Baseline: theta_same=0.6, p_stay_anchor=0.3, count_t0=True
Save CSV: data/outputs/step1.csv

In [ ]:
import pandas as pd

p = Params(theta_same=0.6, p_stay_anchor=0.3, count_t0=True)
df = simulate(p)   # NOTE: your step1 returns only df
df.head()


In [ ]:
total = int(df["footfall"].sum())
expected = p.n_agents * (p.steps + int(p.count_t0))  # use INSTANCE values (not class)
print(f"Total footfall: {total} | Expected: {expected}")
assert total == expected, "Total footfall mismatch!"
print("✅ Simulation totals are correct.")

In [ ]:
def summarize(df: pd.DataFrame) -> pd.DataFrame:
    total = int(df["footfall"].sum())
    anchor_total = int(df.loc[df["role"]=="anchor","footfall"].sum())
    out = pd.DataFrame({
        "metric": ["total","anchor_total","anchor_share",
                   "similar_total","different_total"],
        "value": [
            total,
            anchor_total,
            (anchor_total/total) if total else 0.0,
            int(df.loc[df["category"]=="similar","footfall"].sum()),
            int(df.loc[df["category"]=="different","footfall"].sum())
        ]
    })
    return out

summary = summarize(df)
summary


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12,4))
sns.barplot(data=df.sort_values("node"), x="node", y="footfall", hue="category")
plt.title("Footfall distribution (Step 1)")
plt.xlabel("Node")
plt.ylabel("Footfall")

ensure_dir("data/outputs/figs")
plt.tight_layout()
plt.savefig("data/outputs/figs/footfall_step1.png", dpi=150)
print("[Saved] data/outputs/figs/footfall_step1.png")
plt.show()


## **Sensitivity study**

We vary theta_same ∈ {0.4, 0.6, 0.8} and p_stay_anchor ∈ {0.0, 0.3, 0.5} to observe:

Anchor concentration (anchor_share)

Category totals

Overall traffic unchanged by design (steps × agents [+t0])

In [ ]:
def run_sensitivity(theta_list=(0.4, 0.6, 0.8), stay_list=(0.0, 0.3, 0.5), base=Params()):
    rows = []
    for th in theta_list:
        for ps in stay_list:
            p = Params(
                n_nodes=base.n_nodes, n_anchors=base.n_anchors, steps=base.steps,
                theta_same=th, p_edge=base.p_edge, n_agents=base.n_agents,
                p_stay_anchor=ps, seed=base.seed, count_t0=base.count_t0
            )
            d = simulate(p)
            s = summarize(d).set_index("metric")["value"].to_dict()
            s["theta_same"] = th
            s["p_stay_anchor"] = ps
            rows.append(s)
    out = pd.DataFrame(rows)
    cols = ["theta_same","p_stay_anchor","total","anchor_total","anchor_share","similar_total","different_total"]
    return out[cols].sort_values(["theta_same","p_stay_anchor"]).reset_index(drop=True)

sens = run_sensitivity()
sens


In [ ]:
# Anchor share heatmap-style point plot
plt.figure(figsize=(7,4))
sns.pointplot(data=sens, x="theta_same", y="anchor_share", hue="p_stay_anchor", dodge=True, markers="o")
plt.title("Anchor share vs. threshold & anchor dwell (Step 1)")
plt.ylabel("Anchor share")
plt.xlabel("theta_same")
plt.tight_layout()
plt.savefig("data/outputs/figs/sensitivity_step1_anchor_share.png", dpi=150)
print("[Saved] data/outputs/figs/sensitivity_step1_anchor_share.png")
plt.show()

# Category totals by grid
plt.figure(figsize=(7,4))
sns.pointplot(data=sens, x="theta_same", y="similar_total", hue="p_stay_anchor", dodge=True, markers="o")
plt.title("Similar-category footfall vs. parameters (Step 1)")
plt.ylabel("Similar total")
plt.xlabel("theta_same")
plt.tight_layout()
plt.savefig("data/outputs/figs/sensitivity_step1_similar_total.png", dpi=150)
print("[Saved] data/outputs/figs/sensitivity_step1_similar_total.png")
plt.show()


## **Interpretation** 

vs. Step 0 (random):
Behavior-driven rules yield non-uniform traffic: anchors pull more flow (esp. with higher p_stay_anchor), and homophily (theta_same) fosters clustering.

p_stay_anchor↑ → anchor & neighbors gain share; if too high, diffusion weakens (agents get “stuck”).

theta_same↑ → within-category circulation increases; exploration declines.

Design insight: tune anchor quality and category mix to balance concentration (sales lift) and coverage (exposure).

# threshold: min ratio of same-category neighbors to move toward them
# probability of staying longer at anchor (anchor quality factor)

In [ ]:
def build_mall(params: Params):
    rng = np.random.default_rng(params.seed)
    G = nx.erdos_renyi_graph(params.n_nodes, params.p_edge, seed=params.seed)

    anchors = set(rng.choice(params.n_nodes, params.n_anchors, replace=False))
    for n in G.nodes:
        if n in anchors:
            G.nodes[n]["role"] = "anchor"
            G.nodes[n]["category"] = "similar"
            G.nodes[n]["A"] = 3
        else:
            G.nodes[n]["role"] = "tenant"
            G.nodes[n]["category"] = rng.choice(["similar", "different"])
            G.nodes[n]["A"] = 1
    return G, anchors

## **Movement Logic (Threshold + Anchor Quality)**

1. If on an anchor and rng.random() < p_stay_anchor: stay.

2. Else, compute ratio_same = (# of same-category neighbors) / (# of neighbors).
    - If ratio_same ≥ theta_same and there exists ≥1 same-category neighbor: move to a similar neighbor (uniform among them).

3. Else (fallback): quality-weighted random over (neighbors + here) with weights A.

In [ ]:
def simulate(params: Params):
    rng = np.random.default_rng(params.seed)

    G, anchors = build_mall(params)
    A0 = next(iter(anchors))  
    
    customers = np.full(params.n_agents, A0, dtype=int)  
    footfall = np.zeros(params.n_nodes, dtype=int)

    if params.count_t0:
        for c in customers:
            footfall[c] += 1

    for _ in range(params.steps):
        for i in range(len(customers)):
            here = customers[i]
            neighbors = list(G.neighbors(here))
            if not neighbors:
                neighbors = [here]  # isolation guard

            curr_cat = G.nodes[here]["category"]

            same_neighbors = [v for v in neighbors if G.nodes[v]["category"] == curr_cat]
            ratio_same = len(same_neighbors) / len(neighbors) if neighbors else 0.0

            if G.nodes[here]["role"] == "anchor" and rng.random() < params.p_stay_anchor:
                nxt = here  # dwell at anchor
            elif len(same_neighbors) > 0 and ratio_same >= params.theta_same:
                nxt = int(rng.choice(same_neighbors))  # prefer similar category
            else:
                pool = neighbors + [here]  # allow staying via pool
                weights = np.array([G.nodes[v]["A"] for v in pool], dtype=float)
                probs = weights / weights.sum()
                nxt = int(rng.choice(pool, p=probs))  # quality-weighted random

            customers[i] = nxt
            footfall[nxt] += 1

    df = pd.DataFrame({
        "node": np.arange(params.n_nodes),
        "role": [G.nodes[n]["role"] for n in G.nodes],
        "category": [G.nodes[n]["category"] for n in G.nodes],
        "footfall": footfall
    })
    ensure_dir("data/outputs")
    df.to_csv("data/outputs/step1.csv", index=False)
    print("[Saved] data/outputs/step1.csv")

    return df, G, anchors

In [ ]:
df, G, anchors = simulate(Params())
df.head()